#### 特徴量について検証

In [1]:
data_dir = 'dms_data'
import os
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LinearRegression

2023-09-13 13:28:42.784397: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 13:28:43.972367: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 13:28:43.972906: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 13:28:48.968032: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
#show data_dir items
os.listdir('.')

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'README.md',
 'featureVerification.ipynb',
 'feature_selection_results.txt',
 'hi.ipynb',
 'kanpe.md',
 'miyamotoSenpai.ipynb',
 'neural_network.ipynb',
 'ossPlot1.png',
 'ossPlot2.png',
 'sleepinessPlot1.png',
 'sleepinessPlot2.png',
 'takurasubunrui.ipynb',
 'version.py',
 'withRMSE.txt',
 '機械学習勉強会',
 '過去',
 'dms_data',
 'HELLO.py',
 'graph.ipynb']

In [3]:


def train_and_evaluate(model_type, X_train, y_train, X_test, y_test):
    if model_type == "lgbm":
        train_data = lgb.Dataset(X_train, label=y_train)
        model = lgb.train({'verbose':-1}, train_data)
    elif model_type == "nn":
        model = Sequential()
        model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer=Adam())
        model.fit(X_train, y_train, epochs=10, verbose=0)
    elif model_type == "linear_regression":
        model = LinearRegression().fit(X_train, y_train)
    elif model_type == "svr":
        model = SVR().fit(X_train, y_train)
    elif model_type == "random_forest":
        model = RandomForestRegressor().fit(X_train, y_train)
    
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    
    return model, rmse


In [4]:
def find_best_combination(model_type, combinations, train, test):
    rmses = []
    for feature_comb in combinations:
        _, rmse = train_and_evaluate(model_type, train[feature_comb], train['oss'], test[feature_comb], test['oss'])
        rmses.append(rmse)

    best_comb = combinations[np.argmin(rmses)]
    return best_comb


In [5]:
def read_and_process(directory):
  files = os.listdir(directory)
  df_list = []
  for file in files:
    if file.endswith('.csv'):
      df = pd.read_csv(os.path.join(directory, file))
      df = df.drop(['timestamp'], axis=1)
      df = df.dropna()
      df_list.append(df)
  df_combined = pd.concat(df_list, ignore_index=True)
  
  return df_combined

In [6]:
def show_result():
    # Define directory paths
  train_data_path = data_dir + '/train'
  test_data_path = data_dir + '/test'

  # Read and process data
  train = read_and_process(train_data_path)
  test = read_and_process(test_data_path)

  # Define features
  features = ['m_speed', 'm_speed_var_480', 'm_speed_stddev_480', 'm_acceleration',
              'm_acceleration_var_480', 'm_acceleration_stddev_480', 'm_jerk',
              'm_jerk_var_480', 'm_jerk_stddev_480']

  # Generate all combinations of features
  feature_combinations = []
  for r in range(1, len(features) + 1):
    for subset in itertools.combinations(features, r):
      feature_combinations.append(list(subset))

  # Define models
  models = ["lgbm", "nn", "linear_regression", "svr", "random_forest"]

  # Find the best combination for each model
  for model in models:
    print('model: ' + model)
    best_comb = find_best_combination(model, feature_combinations, train, test)
    print(f"Best feature combination for {model}: {best_comb}")

show_result()


model: lgbm
Best feature combination for lgbm: ['m_speed', 'm_acceleration', 'm_jerk_stddev_480']
model: nn
17/17 [==============================] - 0s 624us/step
Best feature combination for nn: ['m_speed_stddev_480', 'm_acceleration_stddev_480', 'm_jerk', 'm_jerk_var_480', 'm_jerk_stddev_480']
model: linear_regression
Best feature combination for linear_regression: ['m_acceleration_var_480', 'm_acceleration_stddev_480', 'm_jerk_var_480', 'm_jerk_stddev_480']
model: svr
Best feature combination for svr: ['m_acceleration', 'm_acceleration_var_480']
model: random_forest
Best feature combination for random_forest: ['m_speed_var_480', 'm_speed_stddev_480', 'm_acceleration', 'm_acceleration_stddev_480', 'm_jerk_var_480']


In [1]:
data_dir = 'dms_data'
# Define directory paths
train_data_path = data_dir + '/train'
test_data_path = data_dir + '/test'

# Read and process data
train = read_and_process(train_data_path)
test = read_and_process(test_data_path)
features = ['m_speed', 'm_speed_var_480', 'm_speed_stddev_480', 'm_acceleration',
            'm_acceleration_var_480', 'm_acceleration_stddev_480', 'm_jerk',
            'm_jerk_var_480', 'm_jerk_stddev_480']
X_train, y_train, X_test, y_test = train[features], train['oss'], test[features], test['oss']


NameError: name 'data_dir' is not defined

In [7]:
def train_and_evaluate(model_type, X_train, y_train, X_test, y_test):
  if model_type == "lgbm":
      train_data = lgb.Dataset(X_train, label=y_train)
      model = lgb.train({'verbose':-1}, train_data)
  elif model_type == "nn":
      model = Sequential()
      model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
      model.add(Dense(32, activation='relu'))
      model.add(Dense(1))
      model.compile(loss='mean_squared_error', optimizer=Adam())
      model.fit(X_train, y_train, epochs=10, verbose=0)
  elif model_type == "linear_regression":
      model = LinearRegression().fit(X_train, y_train)
  elif model_type == "svr":
      model = SVR().fit(X_train, y_train)
  elif model_type == "random_forest":
      model = RandomForestRegressor().fit(X_train, y_train)
  
  preds = model.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, preds))
  
  return model, rmse

def find_best_combination(model_type, combinations, train, test):
  rmses = []
  for feature_comb in combinations:
      _, rmse = train_and_evaluate(model_type, train[feature_comb], train['oss'], test[feature_comb], test['oss'])
      rmses.append(rmse)

  min_rmse_index = np.argmin(rmses)
  best_comb = combinations[min_rmse_index]
  best_rmse = rmses[min_rmse_index]
  return best_comb, best_rmse

def read_and_process(directory):
  files = os.listdir(directory)
  df_list = []
  for file in files:
      if file.endswith('.csv'):
          df = pd.read_csv(os.path.join(directory, file))
          df = df.drop(['timestamp'], axis=1)
          df = df.dropna()
          df_list.append(df)
  df_combined = pd.concat(df_list, ignore_index=True)

  return df_combined

def with_RMSE():
  data_dir = "./dms_data/"
  # Define directory paths
  train_data_path = data_dir + '/train'
  test_data_path = data_dir + '/test'

  # Read and process data
  train = read_and_process(train_data_path)
  test = read_and_process(test_data_path)

  # Define features
  features = ['m_speed', 'm_speed_var_480', 'm_speed_stddev_480', 'm_acceleration',
              'm_acceleration_var_480', 'm_acceleration_stddev_480', 'm_jerk',
              'm_jerk_var_480', 'm_jerk_stddev_480']

  # Generate all combinations of features
  feature_combinations = []
  for r in range(1, len(features) + 1):
      for subset in itertools.combinations(features, r):
          feature_combinations.append(list(subset))

  # Define models
  models = ["lgbm", "nn", "linear_regression", "svr", "random_forest"]

  # Find the best combination for each model
  for model in models:
      print('model: ' + model)
      best_comb, best_rmse = find_best_combination(model, feature_combinations, train, test)
      print(f"Best feature combination for {model}: {best_comb}")
      print(f"Best RMSE for {model}: {best_rmse}")
with_RMSE()

model: lgbm
Best feature combination for lgbm: ['m_speed', 'm_acceleration', 'm_jerk_stddev_480']
Best RMSE for lgbm: 0.6960560379930254
model: nn
17/17 [==============================] - 0s 601us/step
Best feature combination for nn: ['m_acceleration_var_480', 'm_acceleration_stddev_480', 'm_jerk', 'm_jerk_stddev_480']
Best RMSE for nn: 0.6239647208755708
model: linear_regression
Best feature combination for linear_regression: ['m_acceleration_var_480', 'm_acceleration_stddev_480', 'm_jerk_var_480', 'm_jerk_stddev_480']
Best RMSE for linear_regression: 0.6199633155378697
model: svr
Best feature combination for svr: ['m_acceleration', 'm_acceleration_var_480']
Best RMSE for svr: 0.6267006345046515
model: random_forest
Best feature combination for random_forest: ['m_speed', 'm_acceleration', 'm_acceleration_stddev_480']
Best RMSE for random_forest: 0.7023668023919463


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import itertools
import numpy as np
import pandas as pd
import os
data_dir = './dms_data/'


def train_and_evaluate(model_type, X_train, y_train, X_test, y_test):
  if model_type == "lgbm":
    train_data = lgb.Dataset(X_train, label=y_train)
    model = lgb.train({'verbose': -1}, train_data)
  elif model_type == "nn":
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=Adam())
    model.fit(X_train, y_train, epochs=10, verbose=0)
  elif model_type == "linear_regression":
    model = LinearRegression().fit(X_train, y_train)
  elif model_type == "svr":
    model = SVR().fit(X_train, y_train)
  elif model_type == "random_forest":
    model = RandomForestRegressor().fit(X_train, y_train)

  preds = model.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, preds))

  return model, rmse


def find_best_combination(model_type, combinations, train, test):
  rmses = []
  for feature_comb in combinations:
    _, rmse = train_and_evaluate(model_type, train[feature_comb], train['oss'], test[feature_comb], test['oss'])
    rmses.append(rmse)

  min_rmse_index = np.argmin(rmses)
  best_comb = combinations[min_rmse_index]
  best_rmse = rmses[min_rmse_index]
  return best_comb, best_rmse


def read_and_process(directory, features, scaler=None, fit_scaler=False):
  files = os.listdir(directory)
  df_list = []
  for file in files:
    if file.endswith('.csv'):
      df = pd.read_csv(os.path.join(directory, file))
      df = df.drop(['timestamp'], axis=1)
      df = df.dropna()

      if fit_scaler:
        scaler.fit(df[features])
      if scaler is not None:
        df[features] = scaler.transform(df[features])
      df_list.append(df)
  df_combined = pd.concat(df_list, ignore_index=True)

  return df_combined, scaler


# Define features
features = ['m_speed', 'm_speed_var_480', 'm_speed_stddev_480', 'm_acceleration',
            'm_acceleration_var_480', 'm_acceleration_stddev_480', 'm_jerk',
            'm_jerk_var_480', 'm_jerk_stddev_480']

# Define directory paths
train_data_path = data_dir + '/train'
test_data_path = data_dir + '/test'

# Read and process data
scaler = StandardScaler()
train, scaler = read_and_process(train_data_path, features, scaler, fit_scaler=True)
test, _ = read_and_process(test_data_path, features, scaler)



# Generate all combinations of features
feature_combinations = []
for r in range(1, len(features) + 1):
  for subset in itertools.combinations(features, r):
    feature_combinations.append(list(subset))

# Define models
models = ["lgbm", "nn", "linear_regression", "svr", "random_forest"]

# Find the best combination for each model
for model in models:
  print('model: ' + model)
  best_comb, best_rmse = find_best_combination(model, feature_combinations, train, test)
  print(f"Best feature combination for {model}: {best_comb}")
  print(f"Best RMSE for {model}: {best_rmse}")


ValueError: Found array with 0 sample(s) (shape=(0, 9)) while a minimum of 1 is required by StandardScaler.